# 此為最終版本

In [2]:
!pip install pymongo

     |████████████████████████████████| 380 kB 424 kB/s eta 0:00:01
You should consider upgrading via the '/Users/ian/opt/anaconda3/bin/python -m pip install --upgrade pip' command.


In [3]:
from gensim.models import word2vec
from pymongo import MongoClient
import random
"""
連線至MongoDB資料庫
"""
# client = MongoClient("192.168.217.135", 27017)

# 以下一行, host後面請改成已安裝docker並成功啟動思睿壓模檔案的機器IP
client = MongoClient(host='192.168.217.132', port=27017)
db_auth = client.admin
db_auth.authenticate("root","youtube")
# 連線指定資料庫，資料庫名為db_name
db = client['youtube']

"""
導入推薦系統模型
"""
# 設定欲比對詞彙
KWsearch = input('請輸入欲搜尋標籤: ')

keyword = KWsearch

topic = ['animation', 'beauty', 'entertainment.travel.daily',
         'exercise', 'food', 'game', 'knowledge', 'learn', 
         'news', 'pet', 'technology', 'variatyshow']
#載入12種Model
a = 0
TotalList=[]
maximum = 0
title = ""
checkall = 0
TopicList=[]

for i in range(12):
    try:
        model = word2vec.Word2Vec.load(topic[a] + '.word2vec.model')
        ItemList=[]
        for item in model.wv.most_similar(keyword):
            ItemList.append(item)
#             以下一行可選擇隱藏
#         print('我們在主題: ' + topic[a] + ' 搜尋到您可能喜歡的影片')
        TopicList.append(topic[a])
        a += 1
        TargetList=[]
#             以下兩行可選擇隱藏
#         print(ItemList[0][0])
#         print(ItemList[0][1])
        TargetList.append(ItemList[0][0])
        TargetList.append(ItemList[0][1])
        TotalList.append(TargetList)
        
        listorder = 0
        maximum = TotalList[listorder][1]
        for score in TotalList:
            if TotalList[listorder][1] >= maximum:
                maximum = TotalList[listorder][1]
                
                MostSimilarTopic = ""
                for y in range(len(TopicList[listorder])):
                    en = TopicList[listorder][y]
                    MostSimilarTopic += en
                
                title = ""
                for j in range(len(TotalList[listorder][0])):
                    ch = TotalList[listorder][0][j]
                    title += ch
            listorder += 1

        if a == 11:
            break
    except:
        checkall += 1
#             以下一行可選擇隱藏
#         print('我們在主題: ' + topic[a] + ' 搜尋不到您可能喜歡的影片')
        a += 1

            
if checkall ==12:
    print('\n'+'抱歉我們完全找不到您欲搜尋的影片,請再輸入一次!')
else:
#             以下五行可選擇隱藏
    print('\n')
    print('我們將推薦您 ' + str(MostSimilarTopic) + ' 主題')
    print('我們認為您也喜歡 ' + str(title) + ' 的影片')
    print('該主題與您相似值為: ' + str(maximum))
    print('\n')
    
    MST = MostSimilarTopic
    mycol = db[str(MST)]

    import re
#     title_re = re.compile(title and keyword)
#     title_re = re.compile(keyword and title)
    title_re = re.compile(title)
    z=0
    OutPut=[]
    for x in mycol.find({'Title': title_re}, {"_id": 0, "ChannelLink": 1, 'Title': 1}):
        OutPut.append(x)
#             以下可選擇隱藏
#         print(str(z) + '. ')
#         print(x)
        z += 1
    
    x = random.randint(0,int(z))
    print('\n')
    print('我們推測您也喜歡下面頻道中這則影片: \n' + str(x) + '. ' + OutPut[int(x)]['Title'])
    print('\n')
    print('請進入該頻道搜尋: ' + 'https://www.youtube.com' + OutPut[int(x)]['ChannelLink'])

ServerSelectionTimeoutError: 192.168.217.132:27017: timed out, Timeout: 30s, Topology Description: <TopologyDescription id: 60ebaade32a0f18a76cd257f, topology_type: Single, servers: [<ServerDescription ('192.168.217.132', 27017) server_type: Unknown, rtt: None, error=NetworkTimeout('192.168.217.132:27017: timed out')>]>